In [1]:
#qupath edge detection results didn't format columns in for simple read into dataframe
#so reading lines using tab separator into df
#module using glob to iterate through image files

#backend to be buried away:

#read cell detection data
#return centroid nucleus location as  x,y pixel coordinates
#calculate channel labels as ratios and magnitude relatvie to max. channel value
#tidy column labels

In [4]:
import pandas as pd
from tqdm import tqdm
import glob

**MODULES TO LOOP THROUGH 5 LABEL IMAGE FILES**

In [5]:
#5 label column headers
df_columns = ['Image', 'Name', 'Class', 'Parent', 'ROI', 'Centroid X µm',
       'Centroid Y µm', 'Nucleus: Area', 'Nucleus: Perimeter',
       'Nucleus: Circularity', 'Nucleus: Max caliper', 'Nucleus: Min caliper',
       'Nucleus: Eccentricity', 'Nucleus: EBFP2 Lambda mean',
       'Nucleus: EBFP2 Lambda sum', 'Nucleus: EBFP2 Lambda std dev',
       'Nucleus: EBFP2 Lambda max', 'Nucleus: EBFP2 Lambda min',
       'Nucleus: EBFP2 Lambda range', 'Nucleus: T-sapphire Lambda mean',
       'Nucleus: T-sapphire Lambda sum', 'Nucleus: T-sapphire Lambda std dev',
       'Nucleus: T-sapphire Lambda max', 'Nucleus: T-sapphire Lambda min',
       'Nucleus: T-sapphire Lambda range', 'Nucleus: Venus Lambda mean',
       'Nucleus: Venus Lambda sum', 'Nucleus: Venus Lambda std dev',
       'Nucleus: Venus Lambda max', 'Nucleus: Venus Lambda min',
       'Nucleus: Venus Lambda range', 'Nucleus: mOrange2 Lambda mean',
       'Nucleus: mOrange2 Lambda sum', 'Nucleus: mOrange2 Lambda std dev',
       'Nucleus: mOrange2 Lambda max', 'Nucleus: mOrange2 Lambda min',
       'Nucleus: mOrange2 Lambda range', 'Nucleus: Katushka Lambda mean',
       'Nucleus: Katushka Lambda sum', 'Nucleus: Katushka Lambda std dev',
       'Nucleus: Katushka Lambda max', 'Nucleus: Katushka Lambda min',
       'Nucleus: Katushka Lambda range', 'Nucleus: DRAQ5 Lambda mean',
       'Nucleus: DRAQ5 Lambda sum', 'Nucleus: DRAQ5 Lambda std dev',
       'Nucleus: DRAQ5 Lambda max', 'Nucleus: DRAQ5 Lambda min',
       'Nucleus: DRAQ5 Lambda range']

df = pd.DataFrame(columns=df_columns)

In [6]:
#use glob to iter through image folder
#concat rows from all images into single df

for path in tqdm(glob.glob(r"C:\Users\mrq2226\Desktop\imaging\5_dim_images\*")):
    data = pd.read_csv('5_dim_images\Image_009_measurements.txt', sep='\t', header=None)
    data.columns = data.loc[0]
    data = data.drop(0)
    
    df = df.append(data,sort=False)
    
df = df.reset_index().drop(columns='index')

100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:00<00:00, 20.96it/s]


**READ SINGLE 5 LABEL IMAGE**

In [15]:
df = pd.read_csv('5_dim_images\Image_009_measurements.txt', sep='\t', header=None)

df.columns = df.loc[0]
df = df.drop(0).reset_index().drop(columns='index')

#return values as floats

df = df[df.columns[5:]].copy()
for column in df.columns[2:]:
    df[column] = df[column].apply(lambda x : float(x))
    
df = df.fillna(0)

**PREPROCESS**

In [20]:
#want to define centroid_x,y as pixel rather than distance
#need image size and (pixel size or #pixels 

df['centroid_x'] = df['Centroid X µm']#*image_size/image_dim
df['centroid_y'] = df['Centroid Y µm']#*image_size/image_dim

df['centroid_x'] = df['centroid_x'].apply(lambda x:float(x))
df['centroid_y'] = df['centroid_y'].apply(lambda x:float(x))

In [21]:
#copy df of 5 channel image
df = df[['centroid_x',
        'centroid_y',
        'Nucleus: DRAQ5 Lambda mean',#nucleus
        'Nucleus: EBFP2 Lambda mean',#blue
        'Nucleus: Venus Lambda mean',#green
        'Nucleus: T-sapphire Lambda mean',#cyan
        'Nucleus: mOrange2 Lambda mean',#orange
        'Nucleus: Katushka Lambda mean'#red
        ]].copy()

df = df.rename(columns = {'Nucleus: DRAQ5 Lambda mean':'channel_nucleus',
                         'Nucleus: Venus Lambda mean':'channel_green',
                         'Nucleus: EBFP2 Lambda mean':'channel_blue',
                         'Nucleus: Katushka Lambda mean':'channel_red',
                         'Nucleus: mOrange2 Lambda mean':'channel_orange',
                         'Nucleus: T-sapphire Lambda mean':'channel_cyan'})

In [22]:
#floor values in lowest n_tile to zero for each channel
#could define normal distribution of channel values and floor < 2 SD?

for column in ['channel_blue', 'channel_green', 'channel_cyan', 'channel_orange','channel_red']:
    
    n_tile = 20
    
    def floorValues(x):
        max_value = df[column].max()
        if x < max_value/n_tile:
            return 0
        else:
            return x
        
    df[column] = df[column].apply(lambda x:floorValues(x))

In [23]:
#calcuate channels as ratio of total cell magnitidue
df['sum'] = df['channel_blue']+df['channel_green']+df['channel_cyan']+df['channel_orange']+df['channel_red']

df['ratio_red'] = df['channel_red']/df['sum']
df['ratio_green'] = df['channel_green']/df['sum']
df['ratio_blue'] = df['channel_blue']/df['sum']
df['ratio_cyan'] = df['channel_cyan']/df['sum']
df['ratio_orange'] = df['channel_orange']/df['sum']

#drop all rows where sum is equal to zero
df = df[df['sum'] != 0].reset_index().drop(columns='index')

In [24]:
#calcuate channels as fraction of max channel magnitude
#may be useful for analysing not only ratio of channel labels but magnitude of channel value
df['magnitude_red'] = df['channel_red']/df['channel_red'].max()
df['magnitude_green'] = df['channel_green']/df['channel_green'].max()
df['magnitude_blue'] = df['channel_blue']/df['channel_blue'].max()
df['magnitude_cyan'] = df['channel_cyan']/df['channel_cyan'].max()
df['magnitude_orange'] = df['channel_orange']/df['channel_orange'].max()

In [25]:
#add cell_index for mapping
df = df.reset_index()
df.insert(0, 'cell_index', df['index'].apply(lambda x:'cell_'+str(x)))
df = df.drop(columns='index')

In [26]:
df.to_csv('test files/qupath_measures.csv',index=False)